# 1. 오차역전파(backpropagation)의 필요성

- 매개변수의 기울기를 구할 때 수치 미분을 사용하면
    1. 구현이 편하다.
    2. 계산 시간이 오래 걸린다.
    
- 매개변수의 기울기를 효율적으로 구하기 위해서 오차역전파를 사용

# 2. 계산 그래프

- 복수의 노드와 에지로 표현 

## 2.1. 계산 그래프로 풀기
- 문제 1 : 슈퍼에서 1개에 100원인 사과 2개를 구매. 지불 금액을 구하시요. 소비세는 10%
- 위 문제를 그래프로 표현시
![문제 1](./images/Q1.png)


- 사과의 개수, 소비세를 변수로 취급해 원 밖에 표시
![문제_1변형](./images/Q1.2.png)


- 문제 2 : 슈퍼에서 사과를 2개, 귤을 3개 샀습니다. 사과는 1개에 100원, 귤은 1개 150원 소비세는 10%
![문제_2](./images/Q2.png)


- 계산 그래프를 이용한 문제풀이
    1. 계산 그래프를 구성
    2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다.


- 위와 같은 방법을 순전파라고 합니다.

## 2.2. 국소적 계산

- 자신과 직접 관계된 작은 범위라는 뜻
- 전체 계산이 복잡할지어도 각 노드에서의 계산은 국소적 계산이므로 단순한다.

## 2.3. 왜 계산 그래프로 푸는가?

1. 각 노드에서 문제를 단순화 할 수 있다.
2. 중간 계산 결과를 모두 보관할 수 있다.
3. 미분을 효율적으로 계산할 수 있다.


- 역전파 그림 예시(굵은 화살표 밑의 숫자는 국소적 미분값)
![역전파_예시](./images/BP.png)

# 3. 연쇄법칙

- 국소적 미분을 전달하는 원리


## 3.1. 계산 그래프의 역전파

- y = f(x)의 역전판 그림 예시
![y=f(x)_역전파_예시](./images/BP2.png)


- 역전파의 순서
    1. 국소적 미분을 구한다.
    2. 앞에서 전달된 신호에 국소적 미분을 곱한 뒤 다음 노드로 전달한다.

## 3.2. 연쇄법칙이란?

- 합성 함수

여러 함수로 구성된 함수


- z =(x+y)<sup>2</sup>를 합성 함수로 표현

$$
z = t^2
$$

$$
t = x + y
$$


여기서 합성 함수의 미분에 대한 성질 - 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다. - 는 원리를 사용한다.


- 위의 식을 예시로 설명
$$
\frac{\delta z}{\delta x} = \frac{\delta z}{\delta t} \frac{\delta t}{\delta x}
$$

위의 δt는 서로 지울 수 있으므로

$$
\frac{\delta z}{\delta x} = \frac{\delta z}{\delta x}
$$

가 된다.

# 4. 역전파 

- +, x 등의 연산을 예로 들어 역전파 구조 설명

## 4.1. 덧셈 노드의 역전파

- z = x + y을 해석적으로 계산


$$
\frac{\delta z}{\delta x} = 1 \\
\frac{\delta z}{\delta y} = 1
$$

- 계산 그래프로 표현
![덧셈_계산그래프](./images/plus.png)


- 덧셈 노드의 계산 그래프로 알 수 있는 것
    1. 덧셈 노드의 역전파는 입력된 값을 그대로 다음 노드로 보낸다.
    
## 4.2. 곱셈 노드의 역전파

- z = xy를 해석적으로 미분

$$
\frac{\delta z}{\delta x} = y \\
\frac{\delta z}{\delta y} = x
$$


- 계산 그래프로 표현
![곱셈_계산그래프](./images/multi.png)


- 곱셈 노드의 계산 그래프로 알 수 있는 것
    1. 입력 신호를 서로 바꾸어서 곱한 뒤 다음 노드로 보낸다.
    
## 4.3. 사과 쇼핑 예

- 처음 시작했던 사과 쇼핑에 역전파를 적용하였을때의 계산
![사과_쇼핑_계산그래프](./images/apple_BP.png)

# 5. 단순한 계층 구현하기

- 곱셈 노드를 Mullayer, 덧셈 노드를 Addlayer라는 이름으로 구현한다.\

##  5.1. 곱셈 계층 구현하기

In [1]:
# 곱셈 노드에 대한 구현
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

- 사과 예제에 대한 구현

In [2]:
# 변수 설정
apple = 100
apple_num = 2
tax = 1.1

# layers
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print("순전파를 사용한 사과 값 : {}".format(price))

순전파를 사용한 사과 값 : 220.00000000000003


In [3]:
# backpropagation
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("역전파를 이용한 각 지점의 미분값 : {} {} {}".format(dapple, dapple_num, dtax))

역전파를 이용한 각 지점의 미분값 : 2.2 110.00000000000001 200


##  5.2. 덧셈 계층 구현하기

In [4]:
# 덧셈 노드에 대한 구현
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

# 6. 활성화 함수 계층 구현하기

- Sigmoid, Relu 함수 구현

## 6.1. ReLU 계층

- ReLU 수식
$$
y = 
\cases{
x \quad (x>0) \cr
0 \quad (x\leq0) 
}
$$


- 미분 값
$$
\frac{\delta y}{\delta x} = \cases{
1 \quad (x>0) \cr
0 \quad (x\leq0)
}
$$


- ReLU의 계산 그래프
![ReLU_계산_그래프](./images/ReLU_graph.png)
1. x > 0인경우 미분값이 1이므로 상류에서 내려온 값이 그대로 전달된다.
2. 반대의 경우엔 미분값이 0이므로 0이 내려간다.

In [5]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x, y):
        self.mask = (x <= 0) # x가 0보다 작거나 같으면 True, 아니면 False
        out = x.copy()
        out[self.mask] = 0 # True인 부분은 0으로 만들어 전달한다.
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

## 6.2. Sigmoid 계층

- Sigmoid 수식
$$
y = \frac{1}{1 + exp(-x)}
$$

- Sigmoid 계층의 계산 그래프


![Sigmoid_그래프_순전파](./images/sigmoid.png)
- exp, / 노드 추가

- / 노드 미분 수식
$$
y = \frac{1}{x} \\ \\
\frac{\delta y}{\delta x} = -\frac{1}{x^2} \\ \\
= -y^2
$$


미분값을 보면 상류에서 내려온 값에 -y<sup>2</sup>를 곱하고 내려보내준다.


- exp 미분 수식
$$
\frac{\delta y}{\delta x} = exp(x)
$$
위와 같이 상류에 exp(x)를 곱하고 내려보내준다.


이에 따라서 sigmoid의 모든 노드를 차례차례 보면

1. 단계 / 노드이므로 -y<sup>2</sup>을 곱하여 보내준다.
$$
-\frac{\delta L}{\delta y}y^2
$$
가 된다.


2. 단계 + 노드이므로 상류와 같은 값을 보낸다. 따라서 다음 노드로 내려갈 값은
$$
-\frac{\delta L}{\delta y}y^2
$$
가 된다.


3. 단계 exp 노드이므로 exp(a)를 곱하여 보내준다. a의 값이 -x이므로 내려갈 값은
$$
-\frac{\delta L}{\delta y}y^2exp(-x)
$$
가 된다.

4. 단계 x 노드이므로 순전파의 값을 바꿔 곱해준다. -1을 곱해줘야 하므로 내려갈 값은
$$
\frac{\delta L}{\delta y}y^2exp(-x)
$$
가 된다.

- 간소화한 Sigmoid 계산 그래프
![Sigmoid_그래프_순전파](./images/sigmoid_set.png)

- 정리
$$
y = \frac{1}{1 + exp(-x)} \\
\frac{\delta L}{\delta y}y^2exp(-x) = \frac{\delta L}{\delta y} \frac{1}{(1 + exp(-x))^2}exp(-x) \\
= \frac{\delta L}{\delta y} \frac{1}{1 + exp(-x)} \frac{exp(-x)}{1 + exp(-x)} \\
= \frac{\delta L}{\delta y}y(1-y)
$$


따라서 sigmoid 계층은 y만을 가지고서 역전파가 가능하다.

In [6]:
import numpy as np

# Sigmoid 계층 구현
class SIgmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1.0 - self.out)

# 7 Affine/Softmax 계층 구현하기

- 신경망의 순전파 때 수행하는 행렬 곱을 의미
- numpy에서는 np.dot으로 사용가능

![Affine_계층](./images/affine.png)

- Affine 계층 구현

In [7]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dw = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dw = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

## 7.1 Softmax-with-Loss

- Softmax는 학습을 할때는 필요하지만, 추론을 할 경우에는 가장 높은것 하나만 고르면 되기때문에 softmax를 사용할 필요가 없다.


![소프트맥스_로스](./images/softmax-w-loss.png)

- Softmax-with-Loss 계층 구현 코드

In [8]:
def softmax(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t # 정답 레이블
        self.y = softmax(x) # 결과값
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx